In [1]:
"""
In this file, we will let a human play a game of Lean (using modal).
"""

import io
import json
import os
import pstats
import time

import pexpect

from src.games.lean_game import LeanGame, LeanGameState

informal_prefix = r'''/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
'''
formal_statement = r'''theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
    (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
'''
PROBLEM_STATEMENT = informal_prefix + formal_statement
tactic_state = r'''
-- goal:
-- a r : ℝ
-- u : ℕ → ℝ
-- h₀ : ∀ (k : ℕ), u k = a * r ^ k
-- h₁ : u 1 = 2
-- h₂ : u 3 = 6
-- ⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)
'''

# useful copy+paste for the game.
"""
  -- First, we want to re-write the condition about the second
  -- and fourth terms of the geometric sequence using the definition of a geometric sequence
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  -- Now we can divide the two equations to eliminate $a$ and determine $r$
  have h₃ : r ^ 2 = 3 := by
    nlinarith
  -- Finally, we can substitute back to find $a$
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
  simpa [h₀] using h₄
"""


def send_code_read_json(cmd, timeout_start=30, timeout_finish=30):
    
    child = pexpect.spawn(
        f"{DEFAULT_LAKE_PATH} exe repl",
        cwd=DEFAULT_LEAN_WORKSPACE)

    cmd_json = json.dumps(cmd)
    print(cmd_json)
    child.send(cmd_json + "\r\n")
    # Read the input itself.
    # This should be printed instantly, so timeout is set to 1 second.
    child.expect_exact(cmd_json + "\r\n", timeout=20)
    assert child.after.decode('utf-8') == cmd_json + "\r\n"
    print("Sent code to Lean4 REPL.")

    # Read the output.
    # This code is critical; the repl seems to print out some
    # strange non-json stuff before the actual json output,
    # including characters that delete the previous input,
    # such that it doesn't show up in debug output.
    child.expect_exact("{", timeout=timeout_start)
    res = "{"
    print("Received start of output from Lean4 REPL.")
    # while res is not a valid json string, read lines.
    # All of the lines should print essentially instantly,
    # so there are no timeouts in this loop.
    start_time = time.time()
    while True:
        res = res + child.readline().decode('utf-8')
        try:
            # print all chars in res
            json.loads(res.strip())
            break
        except json.JSONDecodeError as e:
            # print(e)
            pass
        if time.time() - start_time > timeout_finish:
            raise TimeoutError("Lean4 REPL timed out.")
        # time.sleep(0.1)
    
    # kill
    child.close()
    return json.loads(res)


HOME_DIR = os.path.expanduser('~')
DEFAULT_LAKE_PATH = f'{HOME_DIR}/.elan/bin/lake'
DEFAULT_LEAN_WORKSPACE = '../mathlib4/'

LEAN4_DEFAULT_HEADER = "import Mathlib\nimport Aesop\n\nopen BigOperators Real Nat Topology Rat\n\n"



In [2]:

comments = None
with open("../src/sample-data/comments.txt", 'r') as file:
    comments = [line.strip() for line in file.readlines()]

game: LeanGame = LeanGame(
    comment_seeds=comments,
)
state: LeanGameState = game.start_state(
    problem=PROBLEM_STATEMENT,
    tactic_state=tactic_state
)


In [3]:
print("Player Must Act".center(80, "#"))
print(state.human_printout())
# print each possible comment with the index in front.
for i, comment in enumerate(comments):
    print(f"{i}: {comment}")
action = 1 # int(input("Enter your action: "))
# action = comments[action]
# action = input("Enter your action: ")
state = game.next_state(state, action)


################################Player Must Act#################################
-------------------------------------Status-------------------------------------
Fully processed
-------------------------------------Header-------------------------------------
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

------------------------------------Problem-------------------------------------
/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
    (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
------------------------------------Old Code------------------------------------
[Empty Field]
------------------------------------Comment-------------------------------------
[Empty Field]
--------------------------Valid Truncation of

In [4]:

print("LLM Must Act".center(80, "#"))
print(state.human_printout())
print("Enter your code, line-by-line. Type ``` to quit: \n")
# new_code = ""
# while True:
#     line = input()
#     if line == "```":
#         break
#     new_code += line + "\n"
new_code = r"""
  -- First, we want to re-write the condition about the second
  -- and fourth terms of the geometric sequence using the definition of a geometric sequence
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
"""
state.post_LLM_rollout(new_code)


##################################LLM Must Act##################################
-------------------------------------Status-------------------------------------
Just initialized
-------------------------------------Header-------------------------------------
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

------------------------------------Problem-------------------------------------
/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
    (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
------------------------------------Old Code------------------------------------
[Empty Field]
------------------------------------Comment-------------------------------------
-- Start by clearly stating what you need to prove.
[Missi

In [5]:
state.processed = False
print("Lean Verifier Must Act!".center(80, "#"))
print(state.human_printout())
print("Lean4 Input".center(80, "-"))
lean4_input = state.pre_process()

lean4_output = send_code_read_json({
                                        "cmd": lean4_input,
                                        "allTactics": True,
                                        "tactics": True,
                                        "ast": True,
                                        # "env": 0
                                    })

print("Lean4 Output".center(80, "-"))
print(str(lean4_output)[:1000])
print(f"({len(str(lean4_output))} characters)")
state.post_process(lean4_output)

############################Lean Verifier Must Act!#############################
-------------------------------------Status-------------------------------------
Rollout done
-------------------------------------Header-------------------------------------
import Mathlib
import Aesop

set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

------------------------------------Problem-------------------------------------
/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
    (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
------------------------------------Old Code------------------------------------
[Empty Field]
------------------------------------Comment-------------------------------------
-- Start by clearly stating what you need to prove.
[Missing n

Sent code to Lean4 REPL.
Received start of output from Lean4 REPL.


TypeError: spawn.kill() missing 1 required positional argument: 'sig'

In [ ]:
assert False

# Stop here.

In [ ]:
lean4_input

'/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)\n    (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by\n-- Start by clearly stating what you need to prove.\n  -- First, we want to re-write the condition about the second\n  -- and fourth terms of the geometric sequence using the definition of a geometric sequence\n  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,\n    Nat.succ_add]\n'

In [ ]:

lean4_output = send_code_read_json(repl,
                                    {
                                        "cmd": LEAN4_DEFAULT_HEADER + lean4_input,
                                        "allTactics": True,
                                        "tactics": True,
                                        "ast": True,
                                        # "env": 0
                                    })


{"cmd": "import Mathlib\nimport Aesop\n\nopen BigOperators Real Nat Topology Rat\n\n/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : \u211d) (u : \u2115 \u2192 \u211d) (h\u2080 : \u2200 k, u k = a * r ^ k) (h\u2081 : u 1 = 2)\n    (h\u2082 : u 3 = 6) : u 0 = 2 / Real.sqrt 3 \u2228 u 0 = -(2 / Real.sqrt 3) := by\n-- Start by clearly stating what you need to prove.\n  -- First, we want to re-write the condition about the second\n  -- and fourth terms of the geometric sequence using the definition of a geometric sequence\n  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,\n    Nat.succ_add]\n", "allTactics": true, "tactics": true, "ast": true}
Sent code to Lean4 REPL.
Received start of output from Lean4 REPL.


In [ ]:

lean4_output_diff = send_code_read_json(repl,
                                    {
                                        "cmd": lean4_input,
                                        "allTactics": True,
                                        "tactics": True,
                                        "ast": True,
                                        "env": 0
                                    }
)


{"cmd": "/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\frac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : \u211d) (u : \u2115 \u2192 \u211d) (h\u2080 : \u2200 k, u k = a * r ^ k) (h\u2081 : u 1 = 2)\n    (h\u2082 : u 3 = 6) : u 0 = 2 / Real.sqrt 3 \u2228 u 0 = -(2 / Real.sqrt 3) := by\n-- Start by clearly stating what you need to prove.\n  -- First, we want to re-write the condition about the second\n  -- and fourth terms of the geometric sequence using the definition of a geometric sequence\n  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,\n    Nat.succ_add]\n", "allTactics": true, "tactics": true, "ast": true, "env": 0}
Sent code to Lean4 REPL.
Received start of output from Lean4 REPL.


In [ ]:
lean4_output['tactics'], lean4_output['messages']

([{'tactic': 'simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero, Nat.succ_add]',
   'proofState': 1,
   'pos': {'line': 13, 'column': 2},
   'goals': 'a r : ℝ u : ℕ → ℝ h₀ : ∀ (k : ℕ), u k = a * r ^ k h₁ : u 1 = 2 h₂ : u 3 = 6 ⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',
   'endPos': {'line': 14, 'column': 17}}],
 [{'severity': 'error',
   'pos': {'line': 9, 'column': 73},
   'endPos': {'line': 14, 'column': 17},
   'data': 'unsolved goals\na r : ℝ\nu : ℕ → ℝ\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\nh₁ : a * r ^ succ 0 = 2\nh₂ : a * r ^ 3 = 6\n⊢ a * r ^ 0 = 2 / √3 ∨ a * r ^ 0 = -(2 / √3)'}])

In [ ]:
lean4_output

{'tactics': [{'tactic': 'simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero, Nat.succ_add]',
   'proofState': 1,
   'pos': {'line': 13, 'column': 2},
   'goals': 'a r : ℝ u : ℕ → ℝ h₀ : ∀ (k : ℕ), u k = a * r ^ k h₁ : u 1 = 2 h₂ : u 3 = 6 ⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',
   'endPos': {'line': 14, 'column': 17}}],
 'messages': [{'severity': 'error',
   'pos': {'line': 9, 'column': 73},
   'endPos': {'line': 14, 'column': 17},
   'data': 'unsolved goals\na r : ℝ\nu : ℕ → ℝ\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\nh₁ : a * r ^ succ 0 = 2\nh₂ : a * r ^ 3 = 6\n⊢ a * r ^ 0 = 2 / √3 ∨ a * r ^ 0 = -(2 / √3)'}],
 'env': 2,
 'ast': {'tactics': [{'stateBefore': 'a r : ℝ\nu : ℕ → ℝ\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\nh₁ : u 1 = 2\nh₂ : u 3 = 6\n⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',
    'stateAfter': 'a r : ℝ\nu : ℕ → ℝ\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\nh₁ : a * r ^ succ 0 = 2\nh₂ : a * r ^ 3 = 6\n⊢ a * r ^ 0 = 2 / √3 ∨ a * r ^ 0 = -(2 / √3)',
    'pos': 625,
    'endPos':

In [ ]:
lean4_output_diff

{'tactics': [{'tactic': 'simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero, Nat.succ_add]',
   'proofState': 0,
   'pos': {'line': 8, 'column': 2},
   'goals': 'a r : ℝ u : ℕ → ℝ h₀ : ∀ (k : ℕ), u k = a * r ^ k h₁ : u 1 = 2 h₂ : u 3 = 6 ⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',
   'endPos': {'line': 9, 'column': 17}}],
 'messages': [{'severity': 'error',
   'pos': {'line': 4, 'column': 73},
   'endPos': {'line': 9, 'column': 17},
   'data': 'unsolved goals\na r : ℝ\nu : ℕ → ℝ\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\nh₁ : a * r ^ succ 0 = 2\nh₂ : a * r ^ 3 = 6\n⊢ a * r ^ 0 = 2 / √3 ∨ a * r ^ 0 = -(2 / √3)'}],
 'env': 1,
 'ast': {'tactics': [{'stateBefore': 'ℝ : Type u_1\nℕ : Type u_2\na r : ℝ\nu : ℕ → ℝ\nh₀ : ℕ → sorryAx (Sort u_3) true\nh₁ : u (sorryAx ℕ true) = 2\nh₂ : u (sorryAx ℕ true) = 6\n⊢ u (sorryAx ℕ true) = 2 / sorryAx (?m.350 a r u h₀ h₁ h₂) true ∨\n    u (sorryAx ℕ true) = -(2 / sorryAx (?m.353 a r u h₀ h₁ h₂) true)',
    'stateAfter': 'ℝ : Type u_1\nℕ

In [1]:
a = """import Mathlib
import Aesop
import Lean
set_option maxHeartbeats 0

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
    (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  -- First, we want to re-write the condition about the second
  -- and fourth terms of the geometric sequence using the definition of a geometric sequence
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  -- Now we can divide the two equations to eliminate $a$ and determine $r$
  have h₃ : r ^ 2 = 3 := by
    nlinarith
  -- Finally, we can substitute back to find $a$
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
  simpa [h₀] using h₄"""

a

"import Mathlib\nimport Aesop\nimport Lean\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\x0crac{2\\sqrt{3}}{3}$.-/\ntheorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)\n    (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by\n  -- First, we want to re-write the condition about the second\n  -- and fourth terms of the geometric sequence using the definition of a geometric sequence\n  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,\n    Nat.succ_add]\n  have h₁' : a * r = 2 := by simpa [h₀] using h₁\n  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂\n  -- Now we can divide the two equations to eliminate $a$ and determine $r$\n  have h₃ : r ^ 2 = 3 := by\n    nlinarith\n  -- Finally, we can substitute back to find $a$\n  have